# Instrument/Event Map {#sec-inst_event_map}

## Export the Instrument Event Mapping

This shows which instruments are associated with each event in longitudinal projects.

::: {.content-hidden when-format="html"}

## Project Setup

Install and load the necessary packages

In [2]:
#| echo: false
#| output: false
import os
os.environ['R_HOME'] = f'C:/Users/{os.environ.get('USERNAME')}/Miniconda3/envs/r_python_jl/Lib/R'

In [3]:
#| echo: false
#| output: false
%load_ext rpy2.ipython
# only have to run once to allow the R magic command

C:\Users\ekp0303\Miniconda3\envs\r_python_jl\Lib\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


::: {.panel-tabset}

#### R

In [4]:
%%capture 
%%R

library("dplyr")
library("jsonlite")
library("tidyr")
library("REDCapR")
library("knitr")
library("remotes")
library("gt")

In [5]:
%%capture --no-display --no-stdout
%%R

version <- packageVersion("REDCapR")
version

[1] '1.1.9005'


In this project, we will use the bleeding edge version of REDCapR available on Github

In [6]:
%%capture --no-display --no-stdout
%%R

# Detach REDCapR if already loaded, and download the latest version
if (version!='1.1.9005') {
    detach("package:REDCapR", unload=TRUE)
    remotes::install_github("OuhscBbmc/REDCapR")
    library("REDCapR")
    print(packageVersion("REDCapR"))
}

else {
    print("REDCapR package up to date")
}

[1] "REDCapR package up to date"


#### Python

In [7]:
# import pycap
import redcap
import json
import pandas as pd
# import requests

:::

Assign your project URL and Token

::: {.panel-tabset}

#### R

In [8]:
%%R
path = paste0("C:/Users/", Sys.getenv("USERNAME"), '/json_api_data.json')
token <- jsonlite::fromJSON(path)$dev_token$'308'
url <- "https://dev-redcap.doh.wa.gov/api/"

#### Python

In [9]:
path_to_json = f"C:/Users/{os.environ.get('USERNAME')}/json_api_data.json"
api_key = json.load(open(path_to_json))
api_token = api_key['dev_token']['308']
api_url = api_key['dev_url']
project = redcap.Project(api_url, api_token)

:::

:::

::: {.panel-tabset}

#### R

`redcap_event_instruments()`

In [10]:
%%capture
%%R
final_mapping <- redcap_event_instruments(
    redcap_uri = url, 
    token, 
    verbose = TRUE, 
    config_options = NULL
)$data

::: {.content-hidden when-format="html"}

In [11]:
%%R
final_mapping_tbl <- gt(head(final_mapping))
gt::gtsave(final_mapping_tbl, filename = 'export_instrument_event.html', path = "./files/export_files/")

:::

<p align="center"><iframe width="100%" height="270" src="./files/export_files/export_instrument_event.html" title="Quarto Documentation"></iframe></p>

#### Python

`export_instrument_event_mappings()`

In [12]:
project.export_instrument_event_mappings(format_type='df')

,arm_num,unique_event_name,form
0,1,personal_info_arm_1,demographics
1,1,case_intake_arm_1,symptoms
2,1,case_intake_arm_1,test_information
3,1,notifications_arm_1,close_contacts
4,1,notifications_arm_1,work_information


:::

**Note:** Events with multiple instruments (AKA 'form') will occupy multiple rows, with one row per instrument. 

## Export the Repeated Instrument/Event Mapping (PyCap Only)

::: {.panel-tabset}

#### Python

In REDCap projects instruments can be set to repeating and in the case of longitudinal projects, events can be set to repeating. You can use the `export_repeating_instruments_events` function to get a list of repeating events/instruments in your REDCap project. 

In [14]:
project.export_repeating_instruments_events(format_type = 'df')

,event_name,form_name,custom_form_label
0,case_intake_arm_1,NaN,NaN
1,notifications_arm_1,close_contacts,NaN


Note in the data export above that `form_name` is NaN for the 'case_intake_arm_1' event. This means that the entire 'case_intake_arm_1' is repeating. Whereas, in the 'noticications_arm_1' event, the 'close_contacts' form is independently repeating (meaning it can endlessly repeat on it's own within that event). For more informatino on repeating events and isntruments refer to @sec-building_blocks, @sec-definitions and @sec-intro_data_structure.

If a project is not longitudinal, there will be no `event_name` exported. If the project has no repeating instruments or events, the API call will fail and the error string will say: "does not contain any repeating instruments and events"

:::